### Importing all the required libraries

In [45]:
import os
import pandas as pd
!pip install sqlalchemy
import sqlalchemy as sc
from sqlalchemy import create_engine, text
from vertexai.language_models import CodeGenerationModel
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '//Users//sayantrinisaha//Desktop//psyched-metrics-426921-h0-4a9007b858ad.json'

### Reading the test data set on which I will perform some SQL operation

In [47]:
df_penguines = pd.read_csv('penguins.csv')
df_penguines.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


### Creating the table using the dataframe

In [49]:
db_conn = create_engine('sqlite:///:memory:',echo=True)
Penguins_view = df_penguines.to_sql(name='Penguins', con=db_conn)

2024-06-26 17:14:28,059 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-26 17:14:28,066 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2024-06-26 17:14:28,067 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-26 17:14:28,070 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2024-06-26 17:14:28,071 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-26 17:14:28,075 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2024-06-26 17:14:28,076 INFO sqlalchemy.engine.Engine [no key 0.00063s] ()
2024-06-26 17:14:28,077 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2024-06-26 17:14:28,078 INFO sqlalchemy.engine.Engine [no key 0.00062s] ()
2024-06-26 17:14:28,087 INFO sqlalchemy.engine.Engine INSERT INTO "Penguins" ("index", species, island, bill_length_m

### Executing the SQL query using the db_conn

In [51]:
with db_conn.connect() as conn:
    result = conn.execute(text("SELECT * FROM Penguins"))
    result.all()
df_penguines.info()

2024-06-26 17:14:32,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-26 17:14:32,236 INFO sqlalchemy.engine.Engine SELECT * FROM Penguins
2024-06-26 17:14:32,236 INFO sqlalchemy.engine.Engine [generated in 0.00276s] ()
2024-06-26 17:14:32,239 INFO sqlalchemy.engine.Engine ROLLBACK
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


### Creating prefix so that we can feed the SQL generator some prefix of the code and ask to construct other SQL query based on the context

In [53]:
def create_prefix(query):
    prefix = f""" Return a SQL statement that answers the following query:
    {query}

    For a table called 'Penguins' with the following properties:

     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object

     here are some example rows:
     (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE')
     (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
     (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE')

     Only return the SQL statement for the query.
    """
    return prefix

### Creating the function that will use the SQL Query for the question that I will provide through the chat box

In [66]:
def SQL_CodeGenerator():
    #User will provide the input that we will capture
    query = input("Ask a question about the Penguins table")

    #Calling the create_prefix function defined above
    prefix = create_prefix(query)

    #I have used 'code-bison' to generate the code here
    codegen_model = CodeGenerationModel.from_pretrained('code-bison')

    #Passing the prefix to get the resultant SQL query
    print("Hello, I am your AI database assistant")
    print('\n\n')
    result = codegen_model.predict(prefix=prefix)

    return result
SQL_CodeGenerator()    

Ask a question about the Penguins table How many Penguines are there?


Hello, I am your AI database assistant





```sql
SELECT COUNT(*) AS penguin_count
FROM Penguins;
```

### Cleaning up the output resultant query, so that we can execute this query directly

In [70]:
def clean_SQL_Query(sql_statement):
    return sql_statement.replace('```sql','').replace('```','').replace('\n',' ');

### Creating the fucntion that will call the SQL_CodeGenerator function and clean_SQL_Query function and execute the final query and generate the final result.

def nlp_sql_assistant():
    #Calling the SQL_CodeGenerator function
    result = SQL_CodeGenerator()
    
    #Passing the 'result' through clean_SQL_Query function
    sql_comm = clean_SQL_Query(result.text)
    
    with db_conn.connect() as conn:
        result_final = conn.execute(text(sql_comm))
    print('\n\n\n=================================')
    print("here are your results:")
    print(result_final.all())
nlp_sql_assistant()    

### End of code